# Tensorboard
Kode ini bersumber dari modul [Feed Forward](/14_feedforward.ipynb/) dan untuk modul tensorboard ini, kita akan memodifikasi notebook ini. Setiap penambahan yang berbeda pada modul ini dibandingkan modul feedforward akan ditandai dengan menambahkan komentar.

## Persiapan
Sebelum memulai modul ini, anda harus menginstall tensorboard. Berikut perintah yang perlu anda ketikkan di terminal untuk menginstall tensorboard:

**Jika anda menggunakan conda sebagai package manager anda:**
```terminal
conda install -c conda-forge tensorboard
```

**Jika anda tidak menggunakan conda**
```terminal
pip install tensorboard
```

## Menjalankan Tensorboard
Untuk menjalankan tensorboard, anda dapat mengetikkan perintah di bawah ini melalui terminal:

```terminal
tensorboard --logdir=runs
```
Anda dapat mengganti ```runs``` dengan path tensorboard anda apabila anda merasa bahwa tensorboard anda tidak diinstal pada direktori default. Sesudah perintah diatas diketikkan maka akan terdapat url localhost dan port yang dapat digunakan untuk menjalankan tensorboard. Anda dapat mengklik url tersebut sehingga tensorboard terbuka di browser anda. Pada tahap ini, tensorboard memang tidak akan berjalan karena kita belum menjalankan apapun pada program.

Anda dapat mengabaikan keterangan ```TensorFlow installation not found - running with reduced feature set.``` karena memang kita tidak sedang menggunakan tensorflow.

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt


**Mengimpor Tensorboard**
- tensorboard diimpor dengan menggunakan ```from torch.utils.tensorboard import SummaryWriter```
- Selanjutnya kita akan menggunakan ```SummaryWriter``` untuk menulis data ke tensorboard
- Kita membuat sebuah variabel bernama ```writer``` dan kita masukan ```SumaryWriter``` ke dalamnya beserta informasi terkait path dimana data tensorboard akan disimpan.
- Kita juga membutuhkan import sys

In [ ]:
import sys
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("../runs/mnist")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

In [ ]:
train_dataset = torchvision.datasets.MNIST(root='../data',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../data',
                                           train=False,
                                           transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batch_size,
                                           shuffle=False)

In [ ]:
example_batch = iter(train_loader)
samples, labels = example_batch.next()
print(f'Ukuran sample: {samples.shape}')
print(f'Ukuran label: {labels.shape}')

# show 6 samples image with label using subplot
fig = plt.figure(figsize=(10, 4))
for i in range(6):
    plt.subplot(2, 3, i + 1)
    plt.tight_layout()
    plt.imshow(samples[i][0], cmap='gist_yarg')
    plt.title("Digit: {}".format(labels[i]))
    plt.xticks([])
    plt.yticks([])

**Membuat image grid untuk tensorboard**
- Kita membuat sebuah variabel ```sample_grid``` untuk menampung data gambar yang akan ditampilkan pada tensorboard.
- ```writer.add_image``` adalah sebuah method yang digunakan untuk mengirimkan data gambar ke tensorboard.
- ```writer.close()``` adalah sebuah method yang digunakan untuk menutup koneksi ke tensorboard.

In [ ]:
img_grid = torchvision.utils.make_grid(samples)
writer.add_image('Image_Grid_MNIST', img_grid)
writer.close()

**Menjalankan Tensorboard**
- Untuk menjalankan tensorboard, buka terminal dan pindah ke direktori dari repositori ini (pengantarDLpytorch)
- Setelah program ini dijalankan, akan terbentuk folder baru bernama 'runs' (sesuai dengan path pada writer)
- Jalankan tensorboard dengan mengetikkan ```tensorboard --logdir=runs``` (apabila path anda berbeda, silahkan disesuaikan)

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

**Membuat model graph untuk tensorboard**
- Perintah yang digunakan adalah ```writer.add_graph```
- ```writer.add_graph``` adalah sebuah method yang digunakan untuk mengirimkan grafik model ke tensorboard.
- Data sampel yang kita berikan ke writer haruslah direshape terlebih dahulu

In [ ]:
writer.add_graph(model, samples.reshape(-1, 28*28))
writer.close()

Setelah menjalankan bagian diatas, dapat kita amati bahwa terdapat tab baru pada tensorboard yang berjudul 'Graph'. (silahkan refresh halaman untuk melihat perubahan). Anda dapat membuka tab tersebut dan melakukan dobel-klik pada tulisan ```neural_net``` untuk melihat grafik model.

In [ ]:
n_total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_step}], Loss: {loss.item():.4f}')




In [ ]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0

    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    accuracy = 100.0 * n_correct / n_samples
    print(f'Akurasi: {accuracy}')